# 벡터 : chroma vs pinecone
- chroma : 인메모리 vector DB, 로컬메모리 vector DB
- pinecone : 클라우드 vector DB
    * pinecone consoledp API key 생성 -> .env(PINECONE_API_KEY)


# 0. 패키지 설치

In [ ]:
!pip install pinecone-client pinecone-text

# 1. Knowledge Base 구성을 위한 데이터 생성

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

document_list = loader.load_and_split(text_splitter=text_splitter)

len(document_list)

183

In [5]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pc = Pinecone()
index_name = 'tax-index'


c:\Users\Admin\anaconda3\envs\llm-lab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # 데이터를 처음 업로드할 때
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )

In [9]:
# 데이터를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,
    index_name=index_name
)

In [10]:
database

# 2. 답변 생성을 위한 Retrieval 

In [11]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retriever = database.as_retriever(search_kwargs={"k": 3})
retriever.invoke(query)

[Document(id='f15d2cfb-017d-488d-ac50-b5a29f0cf296', metadata={'source': 'tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제9

# 3. 제공되는 prompt를 활용하여 답변 생성

In [12]:
from langchain import hub
prompt = hub.pull('rlm/rag-prompt')

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs={"prompt": prompt},
)

In [20]:
message = qa_chain.invoke({"query":query})
print(message['result'])

연봉 5천만원인 직장인의 소득세는 구체적인 세율과 공제 적용액에 따라 달라지기 때문에 정확한 금액을 제시하기 어렵습니다. 일반적으로 국민세율과 근로소득공제, 기타 공제액을 고려하여 계산해야 합니다. 따라서 정확한 세액 계산을 위해서는 세무 전문가 또는 세금 계산기를 이용하는 것이 좋습니다.
